In [ ]:
import ee
ee.Initialize()

In [ ]:
from geetools import tools, algorithms, collection, ui

In [ ]:
from ipygee import *

In [ ]:
Map = Map()
Map.show()

## BRDF correction in Landsat 8 TOA

In [ ]:
i = ee.Image('LANDSAT/LC8_L1T_TOA_FMASK/LC81780592016030LGN00').select(
    ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B9', 'B10', 'fmask'],
    ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'cirrus', 'thermal', 'fmask']
);

In [ ]:
ic = algorithms.Landsat.brdfCorrect(i)

In [ ]:
p = ee.Geometry.Point([22.0227, 1.4801])
Map.centerObject(p, zoom=10)

In [ ]:
ival = 0.10098563879728317
icval = 0.10012929931321499

In [ ]:
ival_here = tools.image.getValue(i, p, 30, 'client')['blue']

In [ ]:
ival_here == ival

In [ ]:
icval_here = tools.image.getValue(ic, p, 30, 'client')['blue']

In [ ]:
icval_here == icval

In [ ]:
vis = {'bands':['nir', 'swir1','red'], 'min':0, 'max': 0.5}

In [ ]:
Map.addLayer(i, vis, 'original')

In [ ]:
Map.addLayer(ic, vis, 'corrected')

## Using `collection` module

In [ ]:
l8toa = collection.Landsat8TOA()

In [ ]:
vis = l8toa.visualization('NSR')

In [ ]:
p = ee.Geometry.Point([22.21, 1.5])

In [ ]:
col = l8toa.collection.filterBounds(p)

In [ ]:
i2 = col.first()

In [ ]:
proxy = ee.Image.constant(0).rename('proxy')

In [ ]:
colr = col.map(lambda img: l8toa.rename(img)).map(lambda img: img.addBands(proxy))

In [ ]:
colbrdf = colr.map(lambda img: l8toa.brdf(img, True))

In [ ]:
Map.addLayer(i2, vis, 'Landsat 8 TOA')

In [ ]:
i2c = colbrdf.first()

In [ ]:
Map.addLayer(i2c, l8toa.visualization('NSR', True), 'Landsat 8 TOA BRDF Corrected')

In [ ]:
diff = i2.subtract(i2c).multiply(100)

In [ ]:
Map.addLayer(diff, vis.update(dict(min=-1, max=1)), 'BRDF difference')